In [ ]:
import os, glob, json, csv
import numpy as np
import cv2
from checkerboard import *

CHECKERBOARD_DIMS = (10, 7)   # (cols, rows) internal corners
SQUARE_SIZE_MM = 24.0

CALIB_JSON = "calib_4cams_selected40_poseconsistent.json"

CAM_DIRS = {
    "phone":   "../data/intermediate/cameras/calibration/selected_40/phone",
    "olympus": "../data/intermediate/cameras/calibration/selected_40/olympus",
    "flir":    "../data/intermediate/cameras/calibration/selected_40/flir",
    "gray":    "../data/intermediate/cameras/calibration/selected_40/gray",
}
CAM_ORDER = ["olympus", "phone", "flir", "gray"]  # must match your JSON cam_index ordering

REF_CAM = "olympus"

PAIRS = [
    ("olympus", "phone"),
    ("olympus", "flir"),
    ("olympus", "gray"),
]

TARGET_FILENAME = "frame_0071.png"
OUT_DIR = "reproj_results"  

In [ ]:
def make_objp(dims, square_size_mm):
    cols, rows = dims
    objp = np.zeros((cols * rows, 3), np.float32)
    objp[:, :2] = np.mgrid[0:cols, 0:rows].T.reshape(-1, 2) * square_size_mm
    return objp

def solve_pnp(objp, corners, K, dist):
    ok, rvec, tvec = cv2.solvePnP(objp, corners, K, dist, flags=cv2.SOLVEPNP_ITERATIVE)
    if not ok:
        return None, None
    R, _ = cv2.Rodrigues(rvec)
    t = tvec.reshape(3, 1)
    return R, t  # board -> cam

def project_points(objp, R, t, K, dist):
    rvec, _ = cv2.Rodrigues(R)
    imgpts, _ = cv2.projectPoints(objp, rvec, t, K, dist)
    return imgpts  # (N,1,2)

def load_calib(calib_json, cam_order):
    with open(calib_json, "r") as f:
        data = json.load(f)

    intr = {}
    for idx, name in enumerate(cam_order):
        K = np.array(data["intrinsics"][idx]["K"], dtype=np.float64)
        dist = np.array(data["intrinsics"][idx]["dist"], dtype=np.float64).reshape(-1, 1)
        intr[name] = {"K": K, "dist": dist}

    extr = {}
    for item in data["extrinsics_cam0_reference"]:
        idx = item["cam_index"]
        name = cam_order[idx]
        R = np.array(item["R_cam0_to_cam"], dtype=np.float64)
        t = np.array(item["t_cam0_to_cam_mm"], dtype=np.float64).reshape(3, 1)
        extr[name] = {"R_0to": R, "t_0to": t}  # cam0 -> cam
    return intr, extr

def common_basenames(cam_dirs):
    def base_noext(p): return os.path.splitext(os.path.basename(p))[0]
    per = {}
    for name, d in cam_dirs.items():
        files = glob.glob(os.path.join(d, "*.png"))
        per[name] = {base_noext(f): f for f in files}
    common = set.intersection(*[set(m.keys()) for m in per.values()])
    common = sorted(common)
    synced = {name: [per[name][k] for k in common] for name in cam_dirs.keys()}
    return common, synced

def cam0_to_cam(name, extr):
    if name == REF_CAM:
        return np.eye(3), np.zeros((3, 1))
    return extr[name]["R_0to"], extr[name]["t_0to"]

def src_to_tgt_from_cam0(src, tgt, extr):
    R_src0, t_src0 = cam0_to_cam(src, extr)
    R_tgt0, t_tgt0 = cam0_to_cam(tgt, extr)
    R_tgt_src = R_tgt0 @ R_src0.T
    t_tgt_src = t_tgt0 - R_tgt_src @ t_src0
    return R_tgt_src, t_tgt_src

def draw_overlay_no_text(img, corners_detected, corners_reproj):
    out = img.copy()

    # scale marker sizes to image resolution
    h, w = img.shape[:2]
    r = max(2, int(round(0.003 * min(w, h))))   # circle radius in px
    cross = 2 * r                               # half-length of cross arms in px
    thick = max(1, r // 2)                      # line thickness

    # detected: circles
    for p in corners_detected.reshape(-1, 2):
        cv2.circle(
            out,
            (int(round(p[0])), int(round(p[1]))),
            r,
            (0, 255, 0),
            -1
        )

    # reproj: crosses
    for p in corners_reproj.reshape(-1, 2):
        x, y = int(round(p[0])), int(round(p[1]))
        cv2.line(out, (x - cross, y), (x + cross, y), (255, 0, 255), thick)
        cv2.line(out, (x, y - cross), (x, y + cross), (255, 0, 255), thick)

    return out

def write_errors_csv(path, detected, reproj):
    det = detected.reshape(-1, 2)
    rep = reproj.reshape(-1, 2)
    errs = np.linalg.norm(det - rep, axis=1)

    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["i", "det_x", "det_y", "rep_x", "rep_y", "err_px"])
        for i in range(len(errs)):
            w.writerow([i, float(det[i,0]), float(det[i,1]), float(rep[i,0]), float(rep[i,1]), float(errs[i])])
    return errs

def stats_from_errs(errs):
    errs = np.asarray(errs, dtype=np.float64)
    return {
        "num_points": int(errs.size),
        "mean_px": float(np.mean(errs)),
        "median_px": float(np.median(errs)),
        "rms_px": float(np.sqrt(np.mean(errs**2))),
        "max_px": float(np.max(errs)),
    }


In [ ]:
def run_reprojection_for_one_frame(target_filename):
    objp = make_objp(CHECKERBOARD_DIMS, SQUARE_SIZE_MM)
    intr, extr = load_calib(CALIB_JSON, CAM_ORDER)
    keys, synced = common_basenames(CAM_DIRS)

    target_key = os.path.splitext(target_filename)[0]
    if target_key not in keys:
        matches = [k for k in keys if k.endswith(target_key)]
        if len(matches) == 1:
            target_key = matches[0]
        else:
            raise RuntimeError(f"Could not find '{target_filename}' among common frames.")

    frame_idx = keys.index(target_key)

    frame_out_dir = os.path.join(OUT_DIR, target_key)
    os.makedirs(frame_out_dir, exist_ok=True)

    summary = {"frames": {target_key: {"pairs": {}}}}

    for (src, tgt) in PAIRS:
        src_path = synced[src][frame_idx]
        tgt_path = synced[tgt][frame_idx]

        img_src = cv2.imread(src_path, cv2.IMREAD_COLOR)
        img_tgt = cv2.imread(tgt_path, cv2.IMREAD_COLOR)
        if img_src is None or img_tgt is None:
            continue

        corners_src = detect_corners(img_src, CHECKERBOARD_DIMS)
        corners_tgt = detect_corners(img_tgt, CHECKERBOARD_DIMS)
        if corners_src is None or corners_tgt is None:
            continue

        K_src, d_src = intr[src]["K"], intr[src]["dist"]
        K_tgt, d_tgt = intr[tgt]["K"], intr[tgt]["dist"]

        # board -> src pose
        R_srcB, t_srcB = solve_pnp(objp, corners_src, K_src, d_src)
        if R_srcB is None:
            continue

        # src -> tgt from cam0-referenced extrinsics
        R_tgt_src, t_tgt_src = src_to_tgt_from_cam0(src, tgt, extr)

        # board -> tgt predicted
        R_tgtB = R_tgt_src @ R_srcB
        t_tgtB = R_tgt_src @ t_srcB + t_tgt_src

        proj_tgt = project_points(objp, R_tgtB, t_tgtB, K_tgt, d_tgt)

        # save errors + overlay (NO text)
        pair_name = f"{src}_to_{tgt}"
        overlay_path = os.path.join(frame_out_dir, f"{pair_name}_overlay.png")
        errors_path = os.path.join(frame_out_dir, f"{pair_name}_errors.csv")

        errs = write_errors_csv(errors_path, corners_tgt, proj_tgt)
        overlay = draw_overlay_no_text(img_tgt, corners_tgt, proj_tgt)
        cv2.imwrite(overlay_path, overlay)

        stats = stats_from_errs(errs)
        stats.update({
            "src_path": src_path,
            "tgt_path": tgt_path,
            "overlay_png": overlay_path,
            "errors_csv": errors_path,
        })
        summary["frames"][target_key]["pairs"][f"{src}->{tgt}"] = stats

    # save summary.json
    summary_path = os.path.join(OUT_DIR, "summary.json")
    with open(summary_path, "w") as f:
        json.dump(summary, f, indent=2)

    return summary


s = run_reprojection_for_one_frame(TARGET_FILENAME)
frame_key = os.path.splitext(TARGET_FILENAME)[0]
pairs = s["frames"].get(frame_key, {}).get("pairs", {})
for k, v in pairs.items():
    print(k, "mean_px=", round(v["mean_px"], 3), "median_px=", round(v["median_px"], 3), "max_px=", round(v["max_px"], 3))
